In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","12g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time

# get the start time
st = time.time()
url='neo4j://localhost:7692'
password = 'password'

In [3]:
df = spark.read.option('header','true').csv('/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/Dice_12JobTitle.csv',inferSchema=True, escape = '"')
#df = spark.read.option('header','true').csv('/content/drive/MyDrive/Data Science/thesis/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True)

In [4]:
import pyspark.sql.functions as f
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import split, col, monotonically_increasing_id
from pyspark.sql.functions import lower, col
df = df.withColumn("id", monotonically_increasing_id()).withColumn('name', lower(col('name')))

In [5]:
df = df.withColumn("name", f.regexp_replace(f.col("name"), "senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "senior', ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr.", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), ", senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr. ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "jr.", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "mid-senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "lead", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "with", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "remote", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "onsite", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "mid Level", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "associate", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), ", consultant", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "senior Consultant", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "iii", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "ii", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " , ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " intern ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " intern", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "junior ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " junior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "campaign ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "contract: ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "\(.*?\)", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr ", "").alias("name")) \
    .withColumn("name", trim(col("name"))).alias("name")


In [6]:
career = df.select(df.name)
career = career.dropDuplicates(['name']).select('name')
career = career.withColumn("name", ltrim(col("name"))).alias("name")

In [7]:
career.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Career")\
    .save()

In [8]:
df = df.withColumn("timestamp", f.current_timestamp()).alias("timestamp")
jobpost = df.select(df.id, df.timestamp)

In [9]:
jobpost.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "id")\
    .option("labels",":JobPosting")\
    .save()

In [10]:
df.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'ABOUT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Career')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [11]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from transformers import BertForTokenClassification

2023-02-19 01:20:22.825377: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 01:20:23.540882: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-19 01:20:23.540911: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-19 01:20:25.526377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [12]:
from sparknlp.annotator import *
MODEL_NAME = '/home/worker2/Desktop/thesis/NER/NERModel_config'
bert = BertForTokenClassification.loadSavedModel(
     '{}/converting/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

2023-02-19 01:20:28.941536: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-19 01:20:28.991887: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2023-02-19 01:20:28.991970: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-19 01:20:28.992078: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 01:20:29.108181: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:2

In [13]:
bert.write().overwrite().save("./{}".format(MODEL_NAME))

In [14]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

2023-02-19 01:20:48.182173: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 93763584 exceeds 10% of free system memory.


In [15]:
tokenClassifier_loaded.getClasses()

['I-TOOL',
 'B-TOOL',
 'I-KNOW',
 '[SEP]',
 'B-LANG',
 'I-LANG',
 'B-FRAM',
 'I-FRAM',
 'B-KNOW',
 'I-PLAT',
 '[CLS]',
 'O',
 'B-PLAT']

In [16]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('description') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [17]:
df = df.select(df.description.alias('description'))
df = df.withColumn('description', lower(col('description'))) \
        .withColumn('description', f.regexp_replace(col("description"), "[\n\r]", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "®", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "'s", "")) \
        .withColumn('description', f.regexp_replace(col("description"), ";", " ")) \
        .withColumn('description', f.decode('description','UTF-8'))

In [18]:
result = pipeline.fit(df).transform(df)

In [19]:
from pyspark.sql.functions import split, col, monotonically_increasing_id
result = result.withColumn("id", monotonically_increasing_id())

In [20]:
import pyspark.sql.functions as F
result_last = result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [21]:
result_last = result_last.orderBy("id").where(result_last.entity != 'EP]')

In [22]:
programmingLanguage = result_last.distinct().select(result_last.name).where(result_last.entity == 'LANG')
programmingLanguage.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":ProgrammingLanguage")\
    .save()

In [23]:
tool = result_last.distinct().select(result_last.name).where(result_last.entity == 'TOOL')
tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Tool")\
    .save()

In [24]:
framework = result_last.distinct().select(result_last.name).where(result_last.entity == 'FRAM')
framework.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Framework")\
    .save()

In [25]:
platform = result_last.distinct().select(result_last.name).where(result_last.entity == 'PLAT')
platform.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Platform")\
    .save()

In [26]:
knowledge = result_last.distinct().select(result_last.name).where(result_last.entity == 'KNOW')
knowledge.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Knowledge")\
    .save()

In [27]:
jobpost_lo = result_last.distinct().select(result_last.name, result_last.entity,result_last.id)

In [28]:
jobpost_pl = jobpost_lo.select(jobpost_lo.name, jobpost_lo.id).where(jobpost_lo.entity == 'LANG')
jobpost_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'NEED_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [29]:
jobpost_tool = jobpost_lo.select(jobpost_lo.name, jobpost_lo.id).where(jobpost_lo.entity == 'TOOL')
jobpost_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'NEED_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [30]:
jobpost_fram = jobpost_lo.select(jobpost_lo.name, jobpost_lo.id).where(jobpost_lo.entity == 'FRAM')
jobpost_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'NEED_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [31]:
jobpost_plat = jobpost_lo.select(jobpost_lo.name, jobpost_lo.id).where(jobpost_lo.entity == 'PLAT')
jobpost_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'NEED_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [32]:
jobpost_know = jobpost_lo.select(jobpost_lo.name, jobpost_lo.id).where(jobpost_lo.entity == 'KNOW')
jobpost_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'NEED_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [33]:
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 4506.5916039943695 seconds


In [ ]:
spark.stop()